In [1]:
import os
import numpy as np
import pandas as pd
import time
import copy
import datetime
from datetime import datetime

# 忽略警告訊息
import warnings
warnings.filterwarnings( 'ignore' )

###  輸入相關資訊

In [2]:
""" 輸入評估日 """
valuation_date = '2018-06-30'

""" 選擇佣金費用處理方式 """
Method = 'Expensing'     # amortization : 分期攤銷；Expensing：費用化

""" 輸入資料儲存路徑 """
# 前一季產出的資料路徑
dir_data_1 = '/Users/YenLin/Desktop/PAA'    

# 當期保單、保費及賠款資料路徑
dir_data_2 = '/Users/YenLin/Desktop/PAA/Raw_Data/'


# 評估日所屬年度及季別
valuation_year = int( valuation_date[ 0:4 ] )
if ( valuation_date[ 5:7 ] == '03' ) : 
    Quarter = 'Q1'
    previous_Quarter = 'Q4'
    previous_valuation_year = valuation_year - 1 
elif ( valuation_date[ 5:7 ] == '06' ) : 
    Quarter = 'Q2'
    previous_Quarter = 'Q1'
    previous_valuation_year = valuation_year 
elif ( valuation_date[ 5:7 ] == '09' ) : 
    Quarter = 'Q3'  
    previous_Quarter = 'Q2'
    previous_valuation_year = valuation_year 
else : 
    Quarter = 'Q4'  
    previous_Quarter = 'Q3'
    previous_valuation_year = valuation_year 
    
# 前一季產出的資料檔名
previous_data_name = str( previous_valuation_year ) + '_' + previous_Quarter + '_data.txt' 
previous_statement_data_name = str( previous_valuation_year ) + '_' + previous_Quarter + '_statement_data.txt' 
Position_table_name = str( previous_valuation_year ) + '_' + previous_Quarter + '_Position_table.txt'
Performance_table_name = str( previous_valuation_year ) + '_' + previous_Quarter + '_Performance_table.txt'

# 當期保單、保費及賠款資料檔名
policy_data_name = '保單資料_' + str( valuation_year ) + Quarter + '.txt'
premium_data_name = '保費資料_' + str( valuation_year ) + Quarter + '.txt'
loss_data_name = '賠款資料_' + str( valuation_year ) + Quarter + '.txt'

### 讀取資料

In [3]:
# 前一季產出的資料
previous_data = os.path.join( dir_data_1, previous_data_name )
previous_statement_data = os.path.join( dir_data_1, previous_statement_data_name )
Position_table = os.path.join( dir_data_1, Position_table_name )
Performance_table = os.path.join( dir_data_1, Performance_table_name )

# 當期保單及保費資料
policy_data = os.path.join( dir_data_2, policy_data_name )
premium_data = os.path.join( dir_data_2, premium_data_name )
loss_data = os.path.join( dir_data_2, loss_data_name )


print( 'Path of Previous Data: %s' % ( previous_data ) )
print( 'Path of Previous Statement Data: %s' % ( previous_statement_data ) )
print( 'Path of Previous Position Table: %s' % ( Position_table ) )
print( 'Path of Previous Performance Table: %s' % ( Performance_table ) )
print( 'Path of Policy Data: %s' % ( policy_data ) )
print( 'Path of Premium Data: %s' % ( premium_data ) )
print( 'Path of Loss Data: %s' % ( loss_data ) )


# 資料讀取
Previous_data = pd.read_csv( previous_data, encoding = 'utf-8' )
Previous_statement_data = pd.read_csv( previous_statement_data, encoding = 'utf-8' )
Position_table = pd.read_csv( Position_table, encoding = 'utf-8' )
Performance_table = pd.read_csv( Performance_table, encoding = 'utf-8' ) 
Policy_data = pd.read_csv( policy_data, encoding = 'utf-8' )
Premium_data = pd.read_csv( premium_data, encoding = 'utf-8' )
Loss_data = pd.read_csv( loss_data, encoding = 'utf-8' )

Path of Previous Data: /Users/YenLin/Desktop/PAA/2018_Q1_data.txt
Path of Previous Statement Data: /Users/YenLin/Desktop/PAA/2018_Q1_statement_data.txt
Path of Previous Position Table: /Users/YenLin/Desktop/PAA/2018_Q1_Position_table.txt
Path of Previous Performance Table: /Users/YenLin/Desktop/PAA/2018_Q1_Performance_table.txt
Path of Policy Data: /Users/YenLin/Desktop/PAA/Raw_Data/保單資料_2018Q2.txt
Path of Premium Data: /Users/YenLin/Desktop/PAA/Raw_Data/保費資料_2018Q2.txt
Path of Loss Data: /Users/YenLin/Desktop/PAA/Raw_Data/賠款資料_2018Q2.txt


### 當季資料(保單、保費、賠款)的格式調整
- 日期欄位  
- 百分比轉浮點小數

In [4]:
Policy_date_col = [ '簽單日', '保費到期日', '保險起日', '保險迄日' ]
Policy_percentage_col = [ '佣金率' ]
Premium_date_col = [ '保費收取日', '佣金支付日' ]
Loss_date_col = [ '保險起日', '保險迄日', '出險日' ]


for col in Policy_date_col :
    Policy_data[ col ] = Policy_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

for col in Policy_percentage_col :  
    Policy_data[ col ] = Policy_data[ col ].str.strip( '%' ).astype( float ) / 100 

for col in Premium_date_col :
    Premium_data[ col ] = Premium_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

for col in Loss_date_col :
    Loss_data[ col ] = Loss_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

### 檢視當季的保單及保費資料（ 前15筆 ）

In [5]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保單資料筆數 = ' + str( Policy_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保單資料 : ' ) 
Policy_data.head( 15 )

2018 Q2  當季保單資料筆數 = 0

2018 Q2  當季保單資料 : 


,保批單號碼,Group,簽單日,保費到期日,保險起日,保險迄日,簽單保費,佣金率


In [6]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保費資料筆數 = ' + str( Premium_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保費資料 : ' ) 
Premium_data.head( 15 )

2018 Q2  當季保費資料筆數 = 4

2018 Q2  當季保費資料 : 


,保批單號碼,保費收取日,保費收取金額,佣金支付日,佣金支付金額
0,A0003,2018-04-26,1200,2018-04-26,240
1,A0004,2018-05-15,1200,2018-05-15,240
2,A0006,2018-06-28,1200,2018-06-28,240
3,A0009,2018-06-25,1200,2018-06-25,240


In [7]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季賠款資料筆數 = ' + str( Loss_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季賠款資料 : ' ) 
Loss_data.head( 15 )

2018 Q2  當季賠款資料筆數 = 0

2018 Q2  當季賠款資料 : 


,賠案號碼,保批單號碼,Group,保險起日,保險迄日,出險日,已付賠款金額,理賠費用,未決賠款金額


### 檢視前一季產出的資料

In [8]:
print( '前一季保單及保費資料合併後 (前15筆)：' )
Previous_data.head( 15 )

前一季保單及保費資料合併後 (前15筆)：


,保批單號碼,Group,簽單日,保費到期日,保險起日,保險迄日,簽單保費,佣金率,保費收取日,保費收取金額,佣金支付日,佣金支付金額
0,A0001,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0
1,A0002,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0
2,A0003,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,NaN,NaN,NaN,NaN
3,A0004,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,NaN,NaN,NaN,NaN
4,A0005,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-02-28,1200.0,2018-02-28,240.0
5,A0006,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,NaN,NaN,NaN,NaN
6,A0007,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0
7,A0008,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0
8,A0009,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,NaN,NaN,NaN,NaN


In [9]:
print( '前一季用來計算「 財務狀況表 」及「 財務績效表 」的資料 (前15筆)：' )
Previous_statement_data.head( 15 )

前一季用來計算「 財務狀況表 」及「 財務績效表 」的資料 (前15筆)：


,保批單號碼,應付佣金,期初LRC,當期已收保費,當期已付佣金,當期認列之合約收入,取得成本攤銷,期末LRC(amortization),期末LRC(expensing),累積已收保費,累積已付佣金
0,A0001,240.0,0,1200.0,240.0,300.0,60.0,720.0,900.0,1200.0,240.0
1,A0002,240.0,0,1200.0,240.0,300.0,60.0,720.0,900.0,1200.0,240.0
2,A0003,240.0,0,0.0,0.0,300.0,60.0,-240.0,-300.0,0.0,0.0
3,A0004,240.0,0,0.0,0.0,200.0,40.0,-160.0,-200.0,0.0,0.0
4,A0005,240.0,0,1200.0,240.0,200.0,40.0,800.0,1000.0,1200.0,240.0
5,A0006,240.0,0,0.0,0.0,200.0,40.0,-160.0,-200.0,0.0,0.0
6,A0007,240.0,0,1200.0,240.0,100.0,20.0,880.0,1100.0,1200.0,240.0
7,A0008,240.0,0,1200.0,240.0,100.0,20.0,880.0,1100.0,1200.0,240.0
8,A0009,240.0,0,0.0,0.0,100.0,20.0,-80.0,-100.0,0.0,0.0


In [10]:
print( '前一季「 財務狀況表 」( ' + str( Method ) + ' )' )
Position_table

前一季「 財務狀況表 」( Expensing )


,Quarter,現金,合約資產,合約負債,應付佣金,業主權益
0,2018_Q1,4800.0,0,4200.0,960.0,-360.0


In [11]:
print( '前一季「 財務績效表（當季）」( ' + str( Method ) + ' )' )
Performance_table 

前一季「 財務績效表（當季）」( Expensing )


,Quarter,保險合約收入,已發生賠款,佣金費用,保險損益
0,2018_Q1,1800.0,0,2160.0,-360.0


### 合併保單及保費資料
- 當季合併
- 當季與前季合併
- 更新 保費 及 佣金 相關欄位 ( ex: 金額及日期 )
- 保單除列 ？？？

In [12]:
cols_name = [ '保批單號碼', 'Group', '簽單日', '保費到期日', '保險起日', '保險迄日', '簽單保費', '佣金率', '保費收取日', '保費收取金額', '佣金支付日', '佣金支付金額' ]

# 當季合併( 保單及保費 )
current_data = pd.merge( Policy_data, Premium_data, on = [ '保批單號碼' ], how = 'left' )
current_data = current_data[ cols_name ]  # 調整欄位順序

# 當季資料與前一季的資料合併
Data = pd.concat( [ Previous_data, current_data ], axis = 0 )

# 更新前期尚未收取保費的保單
Data.set_index( '保批單號碼', inplace = True )
Premium_data.set_index( '保批單號碼', inplace = True )
Data.update( Premium_data )                                      # 更新保費及佣金相關資料
Data.reset_index( level = '保批單號碼', inplace = True )   # reset index of dataframe
Data_temp = copy.deepcopy( Data )                            # 複製資料

     
date_col_1 = [ '保險起日', '保險迄日']
for col in date_col_1 :
    Data[ col ] = Data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

Data_temp = copy.deepcopy( Data )
    
print( '合併且更新後，資料總筆數 = ' + str( Data.shape[0] ) + '\n' )
print( '檢視合併且更新後的資料 (前15筆) : ' )
Data.head( 15 )

合併且更新後，資料總筆數 = 9

檢視合併且更新後的資料 (前15筆) : 


,保批單號碼,Group,簽單日,保費到期日,保險起日,保險迄日,簽單保費,佣金率,保費收取日,保費收取金額,佣金支付日,佣金支付金額
0,A0001,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0
1,A0002,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0
2,A0003,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-04-26,1200.0,2018-04-26,240.0
3,A0004,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-05-15,1200.0,2018-05-15,240.0
4,A0005,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-02-28,1200.0,2018-02-28,240.0
5,A0006,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-06-28,1200.0,2018-06-28,240.0
6,A0007,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0
7,A0008,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0
8,A0009,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-06-25,1200.0,2018-06-25,240.0


### 計算每張保單的「 保期(月) 」及「 當期已過月數 」

In [13]:
# 定義函數，用以計算月份數
def months( data, evaluate_date ) :
    
    start_year = data[ '保險起日' ].apply( lambda X : X.year )                             # 年度 of 保險起日
    end_year = data[ '保險迄日' ].apply( lambda X : X.year )                              # 年度 of 保險迄日
    start_month = data[ '保險起日' ].apply( lambda X : X.month )                       # 月份 of 保險起日
    end_month = data[ '保險迄日' ].apply( lambda X : X.month )                        # 月份 of 保險迄日
    evaluate_date = datetime.strptime( evaluate_date, '%Y-%m-%d' ).date( )    # 評估日
    evaluate_year = evaluate_date.year                                                          # 評估日所屬年度
    evaluate_month = evaluate_date.month                                                    # 評估日所屬月份
    
    if ( evaluate_month == 3 ) : 
        q_start_date = str( evaluate_year ) + '-' + '01-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期   
    elif ( evaluate_month == 6 ) :
        q_start_date = str( evaluate_year ) + '-' + '04-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期
    elif ( evaluate_month == 9 ) :
        q_start_date = str( evaluate_year ) + '-' + '07-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期
    else :  
        q_start_date = str( evaluate_year ) + '-' + '10-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期
        

    # 計算「 保期(月) 」
    data[ '保期(月)' ] = ( end_year - start_year )*12 + ( end_month - start_month + 1 )
    
    # 計算「 當期已經過月數 」
    data[ '當期已經過月數' ] = np.nan
    for i in np.arange( data.shape[0] ) : 
        first_date = max( q_start_date,  data[ '保險起日' ][ i ] )
        least_date = min( evaluate_date, data[ '保險迄日' ][ i ] ) 
        first_year = first_date.year
        first_month = first_date.month
        least_year = least_date.year
        least_month = least_date.month
        data[ '當期已經過月數' ][ i ] = max( ( least_year - first_year )*12 + ( least_month - first_month + 1 ), 0 )

    return  data.head( 15 )

In [14]:
print( '檢視計算後的資料（前15筆）' )
months( Data, valuation_date )

檢視計算後的資料（前15筆）


,保批單號碼,Group,簽單日,保費到期日,保險起日,保險迄日,簽單保費,佣金率,保費收取日,保費收取金額,佣金支付日,佣金支付金額,保期(月),當期已經過月數
0,A0001,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0,12,3.0
1,A0002,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,2018-01-31,240.0,12,3.0
2,A0003,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-04-26,1200.0,2018-04-26,240.0,12,3.0
3,A0004,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-05-15,1200.0,2018-05-15,240.0,12,3.0
4,A0005,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-02-28,1200.0,2018-02-28,240.0,12,3.0
5,A0006,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-06-28,1200.0,2018-06-28,240.0,12,3.0
6,A0007,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0,12,3.0
7,A0008,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,2018-03-31,240.0,12,3.0
8,A0009,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-06-25,1200.0,2018-06-25,240.0,12,3.0


### 填補及計算下列項目
- (1) 填補「 應付佣金 」 
- (2) 填補「 期初LRC 」               
- (3) 填補「 當期已收保費 」        
- (4) 填補「 當期已付佣金 」       
- (5) 計算「 當期認列之合約收入 」
- (6) 計算「 取得成本攤銷 」         
- (7) 計算「 期末LRC 」               
- (8) 填補「 累積已收保費 」         
- (9) 填補「 累積已付佣金 」         

In [15]:
evaluate_date = datetime.strptime( valuation_date, '%Y-%m-%d' ).date( )    # 文字轉日期
evaluate_year = evaluate_date.year                                                           # 年度 of 評估日
evaluate_month = evaluate_date.month                                                     # 月份 of 評估日

# 評估季的第一天日期
if ( evaluate_month == 3 ) : 
    q_start_date = str( evaluate_year ) + '-' + '01-01'
    q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )     
elif ( evaluate_month == 6 ) :
    q_start_date = str( evaluate_year ) + '-' + '04-01'
    q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   
elif ( evaluate_month == 9 ) :
    q_start_date = str( evaluate_year ) + '-' + '07-01'
    q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  
else :  
    q_start_date = str( evaluate_year ) + '-' + '10-01'
    q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  

# 前期資料的「 保批單號碼 」
Previous_Policy_NO = list( Previous_statement_data[ '保批單號碼' ] )


# 填補「 應付佣金 」及「 期初LRC 」
Data[ '應付佣金' ] = np.nan  
Data[ '期初LRC' ] = np.nan

if  ( Method == 'amortization' ) : 
    for i in np.arange( Data.shape[0] ) :   
        Policy_NO = Data[ '保批單號碼' ][ i ]
        if ( Policy_NO in Previous_Policy_NO ):   
            Data[ '應付佣金' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '應付佣金'  ]
            Data[ '期初LRC' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '期末LRC(amortization)'  ]    
        else : 
            Data[ '應付佣金' ][ i ] = Data[ '簽單保費' ][ i ] * Data[ '佣金率' ][ i ]
            Data[ '期初LRC' ][ i ] = 0
else :
    for i in np.arange( Data.shape[0] ) :   
        Policy_NO = Data[ '保批單號碼' ][ i ]
        if ( Policy_NO in Previous_Policy_NO ):   
            Data[ '應付佣金' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '應付佣金'  ]
            Data[ '期初LRC' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '期末LRC(expensing)'  ]    
        else : 
            Data[ '應付佣金' ][ i ] = Data[ '簽單保費' ][ i ] * Data[ '佣金率' ][ i ]
            Data[ '期初LRC' ][ i ] = 0


# 填補「 當期已收保費 」 及 「 當期已付佣金 」
# 先將「 保費收取日」轉成日期型態，再做運算！
Data[ '當期已收保費' ] = np.nan  
Data[ '當期已付佣金' ] = np.nan  

my_birthday = datetime.strptime( '1983-11-12', '%Y-%m-%d' ).date( )
for i in np.arange( Data.shape[0] ) :
    try:
        if ( Data[ '保費收取日' ][ i ] >= my_birthday ) == True :
            Data[ '保費收取日' ][ i ] = Data[ '保費收取日' ][ i ] 
        else : Data[ '保費收取日' ][ i ] = 'Before my birthday' 
    except : 
        Data[ '保費收取日' ][i] = datetime.strptime( Data[ '保費收取日' ][i], '%Y-%m-%d' ).date( )

for i in np.arange( Data.shape[0] ) :   
    premium_date = Data[ '保費收取日' ][ i ]
    if premium_date >= q_start_date :  # 保費收取日在當季第一天之後
        Data[ '當期已收保費' ][ i ] = Data[ '保費收取金額' ][ i ] 
        Data[ '當期已付佣金' ][ i ] = Data[ '佣金支付金額' ][ i ] 
    else : 
        Data[ '當期已收保費' ][ i ] = 0
        Data[ '當期已付佣金' ][ i ] = 0        

        
# 計算「 當期認列之合約收入 」及「 取得成本攤銷 」
Data[ '當期認列之合約收入' ] =  Data.apply( lambda X : X[ '簽單保費' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )
Data[ '取得成本攤銷' ] = Data.apply( lambda X : X[ '簽單保費' ] * X[ '佣金率' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )


# 計算「 期末LRC 」
Data[ '期末LRC(amortization)' ] = Data.apply( lambda X : X[ '期初LRC' ] + X[ '當期已收保費' ] - X[ '當期已付佣金' ] - X[ '當期認列之合約收入' ] + X[ '取得成本攤銷' ], axis=1 )
Data[ '期末LRC(expensing)' ] = Data.apply( lambda X : X[ '期初LRC' ] + X[ '當期已收保費' ] - X[ '當期認列之合約收入' ], axis=1 )


# 計算「 累積已收保費 」及「 累積已付佣金 」
Data[ '累積已收保費' ] = np.nan
Data[ '累積已付佣金' ] = np.nan
#Previous_statement_data.set_index( '保批單號碼' )
for i in np.arange( Data.shape[0] ) :   
    Policy_NO = Data[ '保批單號碼' ][ i ]
    if ( Data[ '保批單號碼' ][ i ] in Previous_Policy_NO ) : 
        Data[ '累積已收保費' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '累積已收保費'  ] + Data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '當期已收保費'  ]
        Data[ '累積已付佣金' ][ i ] = Previous_statement_data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '累積已付佣金'  ] + Data.loc[ Previous_statement_data[ '保批單號碼' ].str.contains( Policy_NO ), '當期已付佣金'  ]
    else : 
        Data[ '累積已收保費' ][ i ] = Data[ '當期已收保費' ][ i ] 
        Data[ '累積已付佣金' ][ i ] = Data[ '當期已付佣金' ][ i ] 

        
Data.head( 15 )

,保批單號碼,Group,簽單日,保費到期日,保險起日,保險迄日,簽單保費,佣金率,保費收取日,保費收取金額,...,應付佣金,期初LRC,當期已收保費,當期已付佣金,當期認列之合約收入,取得成本攤銷,期末LRC(amortization),期末LRC(expensing),累積已收保費,累積已付佣金
0,A0001,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,...,240.0,900.0,0.0,0.0,300.0,60.0,660.0,600.0,1200.0,240.0
1,A0002,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-01-31,1200.0,...,240.0,900.0,0.0,0.0,300.0,60.0,660.0,600.0,1200.0,240.0
2,A0003,A_2018,2018-01-01,2018-01-01,2018-01-01,2018-12-31,1200,0.2,2018-04-26,1200.0,...,240.0,-300.0,1200.0,240.0,300.0,60.0,420.0,600.0,1200.0,240.0
3,A0004,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-05-15,1200.0,...,240.0,-200.0,1200.0,240.0,300.0,60.0,520.0,700.0,1200.0,240.0
4,A0005,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-02-28,1200.0,...,240.0,1000.0,0.0,0.0,300.0,60.0,760.0,700.0,1200.0,240.0
5,A0006,A_2018,2018-02-01,2018-02-01,2018-02-01,2019-01-31,1200,0.2,2018-06-28,1200.0,...,240.0,-200.0,1200.0,240.0,300.0,60.0,520.0,700.0,1200.0,240.0
6,A0007,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,...,240.0,1100.0,0.0,0.0,300.0,60.0,860.0,800.0,1200.0,240.0
7,A0008,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-03-31,1200.0,...,240.0,1100.0,0.0,0.0,300.0,60.0,860.0,800.0,1200.0,240.0
8,A0009,A_2018,2018-03-01,2018-03-01,2018-03-01,2019-02-28,1200,0.2,2018-06-25,1200.0,...,240.0,-100.0,1200.0,240.0,300.0,60.0,620.0,800.0,1200.0,240.0


### 填報「 財務狀況表 」及「 財務績效表（當季）」

In [16]:
item = str( valuation_year ) + '_' + Quarter 
previous_item = str( previous_valuation_year ) + '_' + previous_Quarter 

select_col = [ '保批單號碼', '應付佣金', '期初LRC', '當期已收保費', '當期已付佣金', '當期認列之合約收入', '取得成本攤銷', '期末LRC(amortization)', '期末LRC(expensing)', '累積已收保費', '累積已付佣金' ]
statement_data = Data[ select_col ]

In [17]:
# 「 財務狀況表 」
Position_table.set_index( 'Quarter', inplace = True )
Position_table.loc[ item ] = np.nan

if  ( Method == 'amortization' ) :
    # 「 現金 」(尚未考慮當期賠款！！)
    Position_table.loc[ item, '現金']  = Position_table.loc[ previous_item, '現金'] + statement_data[ '當期已收保費' ].sum( ) - statement_data[ '當期已付佣金' ].sum( )
    # 「 合約資產 」或「 合約負債 」
    LRC = statement_data[ '期末LRC(amortization)' ].sum( )
    if LRC > 0 :
        Position_table.loc[ item, '合約資產' ] = 0
        Position_table.loc[ item, '合約負債' ] = LRC
    elif LRC < 0 :
        Position_table.loc[ item, '合約資產' ] = -LRC
        Position_table.loc[ item, '合約負債' ] = 0
    else : 
        Position_table.loc[ item, '合約資產' ] = 0 
        Position_table.loc[ item, '合約負債' ] = 0
    
    # 「 業主權益 」= 前期現金 + 合約資產 - 合約負債
    Position_table[ '業主權益' ] = Position_table.apply( lambda X : X[ '現金' ] + X[ '合約資產' ] - X[ '合約負債' ], axis = 1 )

else : 
    # 「 現金 」(尚未考慮當期賠款！！)
    Position_table.loc[ item, '現金' ]  = statement_data[ '累積已收保費' ].sum( ) - statement_data[ '累積已付佣金' ].sum( ) 
    # 「 合約資產 」或「 合約負債 」
    LRC = statement_data[ '期末LRC(expensing)' ].sum( )
    if LRC > 0 :
        Position_table.loc[ item, '合約資產' ] = 0
        Position_table.loc[ item, '合約負債' ] = LRC
    elif LRC < 0 :
        Position_table.loc[ item, '合約資產' ] = -LRC
        Position_table.loc[ item, '合約負債' ] = 0
    else : 
        Position_table.loc[ item, '合約資產' ] = 0 
        Position_table.loc[ item, '合約負債' ] = 0
    
    # 「 應付佣金 」
    Position_table.loc[ item, '應付佣金' ] = statement_data[ statement_data[ '累積已收保費' ] == 0 ][ '應付佣金' ].sum( )
    # 「 業主權益 」= 當期現金 + 合約資產 - 合約負債 - 應付佣金
    Position_table[ '業主權益' ] = Position_table.apply( lambda X : X[ '現金' ] + X[ '合約資產' ] - X[ '合約負債' ] - X[ '應付佣金' ], axis = 1 )


Position_table.reset_index( inplace = True ) 

print( str( valuation_year ) + ' ' + str( Quarter ) + ' 財務狀況表( ' + str( Method ) + ' ) ：' )
Position_table

2018 Q2 財務狀況表( Expensing ) ：


,Quarter,現金,合約資產,合約負債,應付佣金,業主權益
0,2018_Q1,4800.0,0.0,4200.0,960.0,-360.0
1,2018_Q2,8640.0,0.0,6300.0,0.0,2340.0


In [18]:
# 「 財務績效表（當季）」
Performance_table.set_index( 'Quarter', inplace = True )
Performance_table.loc[ item ] = np.nan

if ( Method == 'amortization' ) :
    # 「 保險合約收入 」
    Performance_table.loc[ item, '保險合約收入' ] = statement_data[ '當期認列之合約收入' ].sum( )
    # 「 取得成本攤銷 」
    Performance_table.loc[ item, '取得成本攤銷' ] = statement_data[ '取得成本攤銷' ].sum( )
    # 「 已發生賠款 」
    Performance_table.loc[ item, '已發生賠款' ] = 0 # (尚未考慮當期賠款！！)
    # 「 保險損益 」
    Performance_table[ '保險損益' ] = Performance_table.apply( lambda X : X[ '保險合約收入' ] - X[ '已發生賠款' ] - X[ '取得成本攤銷' ], axis=1 )
else :
    # 「 保險合約收入 」
    Performance_table.loc[ item, '保險合約收入' ] = statement_data[ '當期認列之合約收入' ].sum( )
    # 「 已發生賠款 」
    Performance_table.loc[ item, '已發生賠款' ] = 0 # (尚未考慮當期賠款！！)
    # 「 佣金費用 」
    Performance_table.loc[ item, '佣金費用' ] = 0
    # 「 保險損益 」
    Performance_table[ '保險損益' ] = Performance_table.apply( lambda X : X[ '保險合約收入' ] - X[ '已發生賠款' ] - X[ '佣金費用' ], axis=1 )
    
    
Performance_table.reset_index( inplace = True ) 

print( str( valuation_year ) + ' ' + str( Quarter ) + ' 財務績效表(當季)  ( ' + str( Method ) + ' ) ：')
Performance_table

2018 Q2 財務績效表(當季)  ( Expensing ) ：


,Quarter,保險合約收入,已發生賠款,佣金費用,保險損益
0,2018_Q1,1800.0,0.0,2160.0,-360.0
1,2018_Q2,2700.0,0.0,0.0,2700.0


### 儲存檔案 

In [19]:
Data_temp.to_csv( str( item ) + '_data.txt', index = False, encoding = 'utf-8' ) 
statement_data.to_csv( str( item ) + '_statement_data.txt', index = False, encoding = 'utf-8' ) 
Position_table.to_csv( str( item ) + '_Position_table.txt', index = False, encoding = 'utf-8' ) 
Performance_table.to_csv( str( item ) + '_Performance_table.txt', index = False, encoding = 'utf-8' ) 